In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
with open("input.txt", 'r') as f:
    raw_data = f.read()

In [ ]:
chars = sorted(list(set(raw_data)))
vocab_size = len(chars)

In [ ]:
"".join(chars)

stoi: string to index encoder <br>
itos: index to string decoder

In [ ]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
data = torch.tensor(encode(raw_data), dtype=torch.long)

Splits Data into training-90%, validation-10%

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
#Hyparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 12 # what is the maximum context length for predictions?
max_iters = 300
eval_interval = 100
learning_rate = 1e-3
eval_iters = 100
n_embd = 256
n_head = 8
n_layer = 8
dropout = 0.0
ffn_multiplier = 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Get batches for training and validation

In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

Gives Decay_Mask for single head for a given gamma value

In [ ]:
def get_matrix_D(dim, gamma) -> torch.tensor:
    D = torch.tril(torch.ones(dim, device=device)) #initializes a lower triangular matrix of shape Q*(K.T)
    # Adds gamma values into D
    for i in range(dim[0]): 
        for j in range(i):
            D[i][j] = (gamma)**(i - j)
    return D

Parallel Retention Module

In [ ]:
class SingleHeadRetention(nn.Module):
    def __init__(self, gamma, n_embd, head_dim):
        
        super().__init__()
        self.gamma = gamma
        self.head_dim = head_dim
        self.n_emd = n_embd
        self.W_Q = nn.Linear(n_embd, head_dim, bias=False)
        self.W_K = nn.Linear(n_embd, head_dim, bias=False)
        self.W_V = nn.Linear(n_embd, head_dim, bias=False)
        self.dropout = nn.Dropout(dropout)
         
    def forward(self, X):
        B, T, C = X.shape
        Q = self.W_Q(X) # X @ Q / Q(X)
        K = self.W_K(X) # X @ K
        V = self.W_V(X) # X @ V
        D = get_matrix_D((T,T), self.gamma) # D
        
        retention = torch.einsum("bqh, bkh -> bqk", [Q, K]) # Q @ K.T
        retention = self.dropout(retention)
        retention = torch.einsum("bij, ij -> bij", [retention, D]) # (Q @ K.T) * D
        output = torch.einsum("bij, bjk -> bik", [retention, V]) # ((Q @ K.T) * D) @ V
        return output

In [ ]:
class MultiHeadRetention(nn.Module):
    def __init__(self, n_head, block_size, n_embd):
        super().__init__()
        assert n_embd % n_head == 0
        self.n_head = n_head
        self.n_embd = n_embd
        self.W_G = nn.Linear(n_embd, n_embd, bias=False)
        self.W_O = nn.Linear(n_embd, n_embd, bias=False)
        self.swish = nn.SiLU()
        self.gammas = 1 - 2**(-5-torch.arange(0, n_head))
        # initializes heads for different gamma values
        self.heads = nn.ModuleList([SingleHeadRetention(gamma, n_embd = self.n_embd, head_dim = (self.n_embd // self.n_head)) for gamma in self.gammas])
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, X):
        B, T, C = X.shape
        head_output = []
        # head(i) = Retention(X, gamma(i))
        for head in self.heads:
            head_output.append(head(X))
        # head_output = concat(head(1),..., head(n_head))
        head_output = torch.cat(head_output, dim=-1)
        head_output = self.dropout(head_output)
        # GroupNorm Instance (dividing n_embed into groups of n_head)
        group_norm = nn.GroupNorm(self.n_head, self.n_embd,, device=device)
        # B,T,C -> B*T,C nn.GroupNorm takes input as (N, C)
        Y = head_output.view(B*T, C)
        # applying Group Norm
        Y = group_norm(Y)
        # B*T,C -> B,T,C
        Y = Y.view(B, T, C)
        MHR = self.W_O(self.swish(self.W_G(X)) * Y) # (swish(X @ W_G) * Y) @ W_O
        return MHR
        

In [ ]:
class RetentionBlock(nn.Module):
    def __init__(self, n_head, n_embd, ffn_multiplier):
        super().__init__()
        self.n_head = n_head
        self.n_embd = n_embd
        self.MHR = MultiHeadRetention(self.n_head, block_size, self.n_embd)
        self.FFN = nn.Sequential(
                    nn.Linear(n_embd, ffn_multiplier * n_embd),
                    nn.GELU(),
                    nn.Linear(ffn_multiplier * n_embd, n_embd),
                    nn.Dropout(dropout))
        self.layer_norm_1 = nn.LayerNorm(n_embd)
        self.layer_norm_2 = nn.LayerNorm(n_embd)
        
    def forward(self, X):
        X = X + self.MHR(self.layer_norm_1(X))
        X = X + self.FFN(self.layer_norm_2(X))
        return X

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, n_layer, n_head, block_size, vocab_size, n_embd, ffn_multiplier):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # not using xPos as in paper
        self.layers = nn.Sequential(*[RetentionBlock(n_head, n_embd, ffn_multiplier) for i in range(n_layer)])
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        X = tok_emb + pos_emb 
        X = self.layers(X)
        logits = self.lm_head(X)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # crossEntropy takes (B*T,C) shape inputs 
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
model = LanguageModel(n_layer, n_head, block_size, vocab_size, n_embd, ffn_multiplier)
model = model.to(device)

In [ ]:
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
xb, yb = get_batch('train')

In [ ]:
logits, loss = model(xb, yb)

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
result = model.generate(torch.zeros((1,1), dtype=torch.int, device=device), 300) # Generating new Char from Model, initializing with [[0]]

In [ ]:
decode(result[0].tolist()) # Decoding the result